<a href="https://colab.research.google.com/github/Jineshdhiamr/CRUD-With-Api/blob/main/Product_add_api_ai_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!ngrok authtoken 2qys9u2O4567FcyZbhnTfXkyDAr_3C1HZPhs7iE2v7RyCbokM


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [33]:
!pip install flask ngrok pyngrok spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 101.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [39]:
!pkill -f ngrok
!pkill -f flask


In [40]:
!pkill -f ngrok


In [42]:
from flask import Flask, request, jsonify
import re
import spacy
import threading
from pyngrok import ngrok

nlp = spacy.load("en_core_web_sm")
app = Flask(__name__)

patterns = [
    r"(?:add|insert|store|register|create)?\s*(\d+)\s*(?:units|items|boxes|pieces)?\s*(?:of)?\s*([a-zA-Z\s]+?)\s*(?:with price|at price|for price|costing)?\s*(\d+(?:\.\d{2})?)\s*(?:Rs\.|USD)?\s*(?:in|at)?\s*([a-zA-Z]+(?:Warehouse)?)\s*(?:located in|in|at)?\s*([\w\s]+)",
    r"(?:add|insert|store)?\s*(\d+)\s*(?:units|items)?\s*(?:of)?\s*([a-zA-Z\s]+?)\s*to\s*([a-zA-Z]+(?:Warehouse)?)\s*(?:in|at|located in)?\s*([\w\s]+)\s*for\s*(\d+(?:\.\d{2})?)",
    r"(?:please)?\s*(?:add|store)?\s*(\d+)\s*(?:units)?\s*([a-zA-Z\s]+?)\s*(?:priced at|costing)?\s*(\d+(?:\.\d{2})?)\s*(?:Rs|USD)?\s*(?:to|in)?\s*([a-zA-Z]+(?:Warehouse)?)\s*(?:located in|in)?\s*([\w\s]+)",
    r"put\s*(\d+)\s*([a-zA-Z\s]+?)\s*worth\s*(\d+(?:\.\d{2})?)\s*(?:Rs|USD)?\s*in\s*([a-zA-Z]+(?:Warehouse)?)\s*in\s*([\w\s]+)",
    r"register\s*(\d+)\s*([a-zA-Z\s]+?)\s*(\d+(?:\.\d{2})?)\s*(?:Rs|USD)?\s*at\s*([a-zA-Z]+(?:Warehouse)?)\s*located in\s*([\w\s]+)",
    r"add\s*(\d+)\s*([a-zA-Z\s]+?)\s*(?:costing|with price)?\s*(\d+(?:\.\d{2})?)\s*at\s*([a-zA-Z]+(?:Warehouse)?)\s*at\s*([\w\s]+)",
    r"(\d+)\s*([a-zA-Z\s]+?)\s*with price\s*(\d+(?:\.\d{2})?)\s*(?:Rs|USD)?\s*stored in\s*([a-zA-Z]+(?:Warehouse)?)\s*at\s*([\w\s]+)",
    r"add\s*([a-zA-Z\s]+?)\s*quantity\s*(\d+)\s*price\s*(\d+(?:\.\d{2})?)\s*warehouse\s*([a-zA-Z]+(?:Warehouse)?)\s*location\s*([\w\s]+)",
    r"store\s*(\d+)\s*([a-zA-Z\s]+?)\s*in\s*([a-zA-Z]+(?:Warehouse)?)\s*located\s*in\s*([\w\s]+)\s*at\s*(\d+(?:\.\d{2})?)",
    r"insert\s*(\d+)\s*units of\s*([a-zA-Z\s]+?)\s*worth\s*(\d+(?:\.\d{2})?)\s*into\s*([a-zA-Z]+(?:Warehouse)?)\s*in\s*([\w\s]+)",
    r"(?:add)?\s*([a-zA-Z\s]+?)\s*(\d+)\s*pcs\s*cost\s*(\d+(?:\.\d{2})?)\s*in\s*([a-zA-Z]+(?:Warehouse)?)\s*([\w\s]+)",
    r"add\s*(\d+)\s*([a-zA-Z\s]+?)\s*price\s*(\d+(?:\.\d{2})?)\s*warehouse\s*([a-zA-Z]+)\s*in\s*([\w\s]+)",
    r"store\s*(\d+)\s*([a-zA-Z\s]+?)\s*Rs\.\s*(\d+(?:\.\d{2})?)\s*in\s*([a-zA-Z]+)\s*located at\s*([\w\s]+)",
    r"add\s*(\d+)\s*(?:bottles|boxes)?\s*of\s*([a-zA-Z\s]+?)\s*cost\s*(\d+(?:\.\d{2})?)\s*warehouse\s*([a-zA-Z]+)\s*location\s*([\w\s]+)",
    r"register\s*([a-zA-Z\s]+?)\s*(\d+)\s*units\s*price\s*(\d+(?:\.\d{2})?)\s*in\s*([a-zA-Z]+)\s*in\s*([\w\s]+)",
    r"put\s*(\d+)\s*([a-zA-Z\s]+?)\s*for\s*(\d+(?:\.\d{2})?)\s*in\s*warehouse\s*([a-zA-Z]+)\s*([\w\s]+)",
    r"save\s*(\d+)\s*([a-zA-Z\s]+?)\s*price\s*(\d+(?:\.\d{2})?)\s*in warehouse\s*([a-zA-Z]+)\s*at\s*([\w\s]+)",
    r"create record for\s*(\d+)\s*([a-zA-Z\s]+?)\s*cost\s*(\d+(?:\.\d{2})?)\s*warehouse name\s*([a-zA-Z]+)\s*location\s*([\w\s]+)",
    r"(?:add|insert)?\s*(\d+)\s*of\s*([a-zA-Z\s]+?)\s*in\s*([a-zA-Z]+)\s*at\s*([\w\s]+)\s*price\s*(\d+(?:\.\d{2})?)",
    r"add\s*item\s*([a-zA-Z\s]+?)\s*qty\s*(\d+)\s*in\s*([a-zA-Z]+)\s*([\w\s]+)\s*cost\s*(\d+(?:\.\d{2})?)"
    # Add more patterns as needed for edge cases
]

def extract_all_entities(text):
    doc = nlp(text)
    product = None
    price = None
    warehouse_name = None
    warehouse_location = None
    quantity = None

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            groups = match.groups()
            nums = [g for g in groups if g and g.strip().isdigit()]
            words = [g.strip() for g in groups if g and not g.strip().isdigit() and not g.strip().replace('.', '', 1).isdigit()]

            quantity = int(nums[0]) if nums else None
            price = nums[1] if len(nums) > 1 else None
            product = words[0] if words else None
            warehouse_name = words[1] if len(words) > 1 else None
            warehouse_location = words[2] if len(words) > 2 else None
            break

    for ent in doc.ents:
        if ent.label_ == "MONEY" and not price:
            price = re.sub(r"[^\d.]", "", ent.text)
        elif ent.label_ == "ORG" and not warehouse_name:
            warehouse_name = ent.text
        elif ent.label_ == "GPE" and not warehouse_location:
            warehouse_location = ent.text

    return {
        "product": product or "Unknown Product",
        "price": price or "Unknown Price",
        "warehouse_name": warehouse_name or "Unknown Warehouse",
        "warehouse_location": warehouse_location or "Unknown Location",
        "quantity": quantity or 1
    }

@app.route('/parse-product', methods=['POST'])
def parse_product():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided."}), 400

    entities = extract_all_entities(text)

    return jsonify(entities)

@app.route('/add-product', methods=['POST'])
def add_product():
    data = request.get_json()
    required = ["product_name", "price", "warehouse_name", "quantity"]
    if not all(k in data and data[k] for k in required):
        return jsonify({"error": "Missing required fields."}), 400

    return jsonify({
        "message": "Product added successfully.",
        "product_name": data["product_name"],
        "price": data["price"],
        "warehouse_name": data["warehouse_name"],
        "quantity": data["quantity"]
    })

def run_app():
    app.run(host="0.0.0.0", port=5000)

public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")
threading.Thread(target=run_app).start()


🚀 Public URL: NgrokTunnel: "https://8f97-34-67-92-67.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
